### Importações

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, RocCurveDisplay, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder


### Carregar Datasets

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/PS-Ligia_Time16/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
'''
Se você executou o notebook de EDA deste projeto, e portanto, gerou e salvou os
datasets processados; Descomente este código abaixo e fique a vontade para
utilizar os datasets que foram salvos no seu próprio Google Drive.

Caso não tenha criado os datasets, utilize o trecho descomentado para carregar
ele já pronto do nosso Drive.
'''
# df_completo = pd.read_csv(os.path.join(output_path, 'risco_cardiovascular_features.csv'))

df_completo = pd.read_csv("https://drive.google.com/uc?id=142P-9xy-kRboRVpHj2t9OniS0DUF0QTp")

In [ ]:
coluna_alvo = 'BP_Category'

# Carrega todas as features e o alvo diretamente do arquivo de features
print("\n=== Informações do DataSet ===")
print(f"Shape df_completo: {df_completo.shape}")
print("\nValores nulos em df_completo:")
print(df_completo.isnull().sum())
print("\nTipos de dados:")
print(df_completo.dtypes)
display(df_completo.head())



### Pré-Processamento dos Dados

In [ ]:
print("Colunas finais (após carregamento): ", df_completo.columns.tolist())

colunas_categoricas = df_completo.select_dtypes(include=['object']).columns.tolist()

print("\nColunas categóricas antes da codificação:", colunas_categoricas)

# Aplicar LabelEncoder a todas as colunas categóricas identificadas
for coluna in colunas_categoricas:
    le = LabelEncoder()
    df_completo[coluna] = le.fit_transform(df_completo[coluna])
    print(f"Coluna '{coluna}' codificada.")

print("\nApós codificação:")
print(df_completo.dtypes)
display(df_completo.head())




### Divisão de Dados e Normalização

In [ ]:
# Separar x (features) e y (target)
x = df_completo.drop(coluna_alvo, axis=1)
y = df_completo[coluna_alvo] # Agora y será uma única Series com valores numéricos

# Dividir treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalizar
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

print(f"\n Dados prontos para treinar")
print(f"x_train shape: {x_train_scaled.shape}")
print(f"x_test shape: {x_test_scaled.shape}")

# Treinar o modelo



### Definição de Hiperparâmetros para GridSearch

In [ ]:
parametros = {
    'C': [0.01, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'class_weight': [None, 'balanced'],
    'penalty': ['l2']
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=5000),
    parametros,
    cv=5,
    scoring='accuracy',
    verbose=1
)

grid_search.fit(x_train_scaled, y_train)

#resultados do grid
print("="*60)
print(f"✅ Melhores parâmetros: {grid_search.best_params_}")
print(f"✅ Melhor acurácia no CV: {grid_search.best_score_:.4f}")
print("="*60)



### Resultados

In [ ]:
modelo = grid_search.best_estimator_
y_pred = modelo.predict(x_test_scaled)

print("\nAcurácia:", accuracy_score(y_test, y_pred))

nomes_classes = ['Elevada', 'Normal', 'Hipertensão']
print("\nRelatório:")
print(classification_report(y_test, y_pred, target_names=nomes_classes))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)

print("\nMatriz de Confusão:")
print(cm)